In [6]:
%pip install langchain-community langchain-openai langchain-ollama colorama faiss-cpu unstructured

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain-community in c:\users\rahul\documents\personal_learning\genai llm\.venv\lib\site-packages (0.3.14)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\rahul\documents\personal_learning\genai llm\.venv\lib\site-packages (4.48.0)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install langchain huggingface_hub sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install libmagic


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os

if 'OPENAI_API_KEY' in os.environ:
    print("OPENAI_API_KEY is set")
else:
    print("OPENAI_API_KEY is not set")

OPENAI_API_KEY is set


### Building the RAG Assistant
Step 1: Create the Index Knowledge Base (index_knowledge_base.py)

In [11]:
%%writefile index_knowledge_base_sql.py

import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from colorama import Fore, Style, init

# Initialize colorama for terminal color support
init(autoreset=True)

def create_index():
    openai_api_key = os.environ.get('OPENAI_API_KEY')  # Replace with your actual API key
    try:
        print(f"{Fore.LIGHTBLUE_EX}Loading documents from the 'rag_code_assistant/sample-sql-scripts-main' folder...{Style.RESET_ALL}")
        text_loader_kwargs = {"autodetect_encoding": True}
        loader = DirectoryLoader("rag_code_assistant", glob="*/Scripts/*/*.sql", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
        print(loader)
        documents = loader.load()
        print(f"{Fore.LIGHTGREEN_EX}Documents loaded successfully!{Style.RESET_ALL}")

        print(f"{Fore.LIGHTBLUE_EX}Indexing documents...{Style.RESET_ALL}")
        data_store = FAISS.from_documents(documents, OpenAIEmbeddings(openai_api_key=openai_api_key))
        data_store.save_local("index_sql")
        print(f"{Fore.LIGHTGREEN_EX}Knowledge base indexed successfully!{Style.RESET_ALL}")
    except Exception as e:
        print(f"{Fore.RED}Error in creating index: {e}{Style.RESET_ALL}")



Overwriting index_knowledge_base_sql.py


In [12]:
!python index_knowledge_base_sql.py

Step 2: Create the Retrieval Pipeline (retrieval_pipeline.py)

In [13]:
%%writefile retrieval_pipeline_sql.py

import os
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_ollama.llms import OllamaLLM
from colorama import Fore, Style, init
import asyncio

# Initialize colorama for terminal color support
init(autoreset=True)

openai_api_key = os.environ.get('OPENAI_API_KEY')  # Replace with your actual API key

try:
    print(f"{Fore.LIGHTYELLOW_EX}Loading FAISS index...{Style.RESET_ALL}")
    data_store = FAISS.load_local(
        "index_sql",
        OpenAIEmbeddings(openai_api_key=openai_api_key),
        allow_dangerous_deserialization=True
    )
    print(f"{Fore.LIGHTGREEN_EX}FAISS index loaded successfully!{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}Error loading FAISS index: {e}{Style.RESET_ALL}")

try:
    print(f"{Fore.LIGHTYELLOW_EX}Initializing the Ollama LLM...{Style.RESET_ALL}")
    llm = OllamaLLM(model="llama3.2", base_url="http://localhost:11434", timeout=120)
    print(f"{Fore.LIGHTGREEN_EX}Ollama LLM initialized successfully!{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}Error initializing Ollama LLM: {e}{Style.RESET_ALL}")

def query_pipeline(query):
    try:
        print(f"{Fore.LIGHTBLUE_EX}Searching the knowledge base...{Style.RESET_ALL}")
        context = data_store.similarity_search(query)
        print(f"{Fore.LIGHTGREEN_EX}Context found:{Style.RESET_ALL} {context}")

        prompt = f"Context: {context}\nQuestion: {query}. Response only if context is relevant to calculator code, else response sorry cannot help you in this context."
        print(f"{Fore.LIGHTBLUE_EX}Sending prompt to LLM...{Style.RESET_ALL}")
        response = llm.generate(prompts=[prompt])
        print(f"{Fore.LIGHTGREEN_EX}Received response from LLM.{Style.RESET_ALL}")
        return response
    except Exception as e:
        print(f"{Fore.RED}Error in query pipeline: {e}{Style.RESET_ALL}")
        return f"{Fore.RED}Unable to process the query. Please try again.{Style.RESET_ALL}"

Overwriting retrieval_pipeline_sql.py


Step 3: Create the Interactive Assistant (interactive_assistant.py)

In [14]:
%%writefile interactive_assistant_sql.py

from retrieval_pipeline_sql import query_pipeline
from colorama import Fore, Style, init

# Initialize colorama for terminal color support
init(autoreset=True)

def interactive_mode():
    print(f"{Fore.LIGHTBLUE_EX}Start querying your RAG assistant. Type 'exit' to quit.{Style.RESET_ALL}")
    while True:
        try:
            query = input(f"{Fore.LIGHTGREEN_EX}Query: {Style.RESET_ALL}")

            if query.lower() == "exit":
                print(f"{Fore.LIGHTBLUE_EX}Exiting... Goodbye!{Style.RESET_ALL}")
                break

            print(f"{Fore.LIGHTYELLOW_EX}Processing your query...{Style.RESET_ALL}")
            response = query_pipeline(query)
            print(f"{Fore.LIGHTCYAN_EX}Assistant Response: {Style.RESET_ALL}{response.generations[0][0].text}")

        except Exception as e:
            print(f"{Fore.RED}Error in interactive mode: {e}{Style.RESET_ALL}")

Overwriting interactive_assistant_sql.py


Step 4: Create the Main Script (main.py)


In [15]:
%%writefile main_sql.py 

from interactive_assistant_sql import interactive_mode
from index_knowledge_base_sql import create_index
from colorama import Fore, Style, init

# Initialize colorama for terminal color support
init(autoreset=True)

print(f"{Fore.LIGHTBLUE_EX}Initializing the RAG-powered Code Assistant...{Style.RESET_ALL}")

try:
    print(f"{Fore.LIGHTYELLOW_EX}Indexing the knowledge base...{Style.RESET_ALL}")
    create_index()
    print(f"{Fore.LIGHTGREEN_EX}Knowledge base indexed successfully!{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}Error while indexing the knowledge base: {e}{Style.RESET_ALL}")

try:
    print(f"{Fore.LIGHTYELLOW_EX}Starting the interactive assistant...{Style.RESET_ALL}")
    interactive_mode()
except Exception as e:
    print(f"{Fore.RED}Error in interactive assistant: {e}{Style.RESET_ALL}")

Overwriting main_sql.py
